In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
sys.path.append(os.getcwd() + '/..')
import src

# Experiment setup
* Here we basically just load the game and check that it works

In [3]:
from __future__ import print_function 
import matplotlib.pyplot as plt
import numpy as np
import gym
from tqdm import tqdm

%matplotlib inline
%env THEANO_FLAGS="floatX=float32"

env: THEANO_FLAGS="floatX=float32"


In [4]:
# np.random.seed(seed=20)
# np.random.seed(seed=42)


In [5]:
GAME = "FrontoPolarStocks-v0"

N_AGENTS = 1
SEQ_LENGTH = 25

In [6]:
env = gym.make(GAME)
env.reset()

action_shape = (env.action_space.num_discrete_space,)
action_emb_shape = (*action_shape, 3)
state_shape = env.observation_space.shape

random_action = np.random.randint(low=0, high=3, size=action_shape)

state, _, _, _ = env.step(random_action)

action_names = np.array(["sell", "pass", "buy"]) #i guess so... i may be wrong

print(state)

[2017-09-05 00:00:27,258] Making new env: FrontoPolarStocks-v0


[ 136.010002   17.64    ]


# Basic agent setup
Here we define a simple agent that maps game images into Qvalues using shallow neural network.


In [7]:
import theano
from theano import tensor as T
import lasagne

theano.config.exception_verbosity = 'high'
theano.config.optimizer = 'fast_compile'

In [8]:
from agent.agent import build_agent

In [9]:
agent, action_layer, V_layer = build_agent(action_emb_shape, state_shape)

In [10]:
agent.action_layers[0].output_shape

(None, 2)

In [11]:
#Since it's a single lasagne network, one can get it's weights, output, etc
weights = lasagne.layers.get_all_params((action_layer,V_layer),trainable=True)

# Create and manage a pool of atari sessions to play with

* To make training more stable, we shall have an entire batch of game sessions each happening independent of others
* Why several parallel agents help training: http://arxiv.org/pdf/1602.01783v1.pdf
* Alternative approach: store more sessions: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [12]:
from agentnet.experiments.openai_gym.pool import EnvPool

pool = EnvPool(agent, GAME, N_AGENTS, max_size=10000)

[2017-09-05 00:00:28,805] Making new env: FrontoPolarStocks-v0


In [13]:
_,action_log,reward_log,_,_,_  = pool.interact(10)


print(action_names[action_log])
print(reward_log)

[[['buy' 'sell']
  ['sell' 'buy']
  ['buy' 'buy']
  ['buy' 'buy']
  ['buy' 'buy']
  ['sell' 'buy']
  ['buy' 'buy']
  ['sell' 'buy']
  ['buy' 'sell']
  ['pass' 'buy']]]
[[ -1.72505901  -0.56255999  -1.78435997   2.40297003   9.784077
    1.67338601  17.08936      5.827854    -3.28881197   0.        ]]


In [14]:
#load first sessions (this function calls interact and remembers sessions)
pool.update(SEQ_LENGTH)

# a2c loss

Here we define obective function for actor-critic (one-step) RL.

* We regularize policy with expected inverse action probabilities (discouraging very small probas) to make objective numerically stable


In [15]:
#get agent's Qvalues obtained via experience replay
replay = pool.experience_replay.sample_session_batch(100, replace=True)

_,_,_,_,(policy_seq,V_seq) = agent.get_sessions(
    replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,
)


In [16]:
from agent.train_step import get_train_step

# replay.rewards = T.printing.Print(">>>>>>>>>> replay.rewards: ")(replay.rewards)

train_step = get_train_step(policy_seq, V_seq, weights, replay)

# Demo run

In [17]:
untrained_reward = pool.evaluate(save_path="./records", record_video=True)

[2017-09-05 00:00:32,977] Making new env: FrontoPolarStocks-v0
[2017-09-05 00:00:33,022] Clearing 4 monitor files from previous run (because force=True was provided)
[2017-09-05 00:00:33,024] Starting new video recorder writing to /home/manatee/Desktop/inn.prac/Algorithms/src/records/openaigym.video.0.2122.video000000.mp4
[2017-09-05 00:00:36,647] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/manatee/Desktop/inn.prac/Algorithms/src/records')


Episode finished after 200 timesteps with reward=14311.205137728051


In [18]:
# from IPython.display import HTML

# video_path="records/openaigym.video.0.29209.video000000.mp4"

# HTML("""
# <video width="640" height="480" controls>
#   <source src="{}" type="video/mp4">
# </video>
# """.format(video_path))

# Vime

In [19]:
from agent.bnn import BNN
# print("action_emb_shape: ", action_emb_shape)
bnn = BNN(state_shape, action_shape, action_emb_shape, replay)

A:  (None, 2)
A_emb:  (None, 6)
S:  (None, 2)
Concat:  (None, 8)
l_dense:  :  (None, 8)  ->  (None, 50)
l_out:  :  (None, 50)  ->  (None, 2)


# Training loop

In [20]:
#starting epoch
epoch_counter = 1

#full game rewards
rewards = {epoch_counter: untrained_reward}

In [21]:
#pre-fill pool
for i in tqdm(range(100)):
# for i in tqdm(range(10)):
    pool.update(SEQ_LENGTH, append=True, preprocess=bnn.add_vime_reward)
#     pool.update(SEQ_LENGTH, append=True)

100%|██████████| 100/100 [00:06<00:00, 16.72it/s]


In [22]:
#pre-train BNN (mitigate training lag on first iterations where BNN is stupid)

for i in tqdm(range(100)):
# for i in tqdm(range(10)):
    from_pool = bnn.sample_from_pool()
    bnn.train_step(*from_pool)

100%|██████████| 100/100 [00:00<00:00, 204.96it/s]


In [23]:
#the loop may take eons to finish.
#consider interrupting early.
loss = 0

for i in tqdm(range(1000)):
# for i in tqdm(range(20)):
# for i in tqdm(range(1)):
    #train
    for i in range(10):
        pool.update(SEQ_LENGTH, append=True, preprocess=bnn.add_vime_reward)

    for i in range(10):
        new_loss = train_step()
        loss = loss * 0.99 + new_loss * 0.01
    
    for i in range(10):
        bnn.train_step(*bnn.sample_from_pool())

#     if epoch_counter % 100 == 0:
#         print("Loss: ", loss)

    if epoch_counter % 100 == 0:
        #average reward per game tick in current experience replay pool
        pool_mean_reward = np.average(pool.experience_replay.rewards.get_value()[:,:-1],
                                      weights=1 + pool.experience_replay.is_alive.get_value()[:,:-1])
        pool_size = pool.experience_replay.rewards.get_value().shape[0]
        print("iter=%i\treward/step=%.5f\tpool_size=%i\tvime ma=%.5f"%(epoch_counter,
                                                         pool_mean_reward,
                                                         pool_size,
                                                         bnn.vime_reward_ma))
        

    ##record current learning progress and show learning curves
    if epoch_counter % 500 == 0:
        n_games = 10
        rewards[epoch_counter] = pool.evaluate(
            record_video=False,
            n_games=n_games,
            verbose=False)
        print("Current score(mean over %i) = %.3f"%(n_games,np.mean(rewards[epoch_counter])))
    
    
    epoch_counter  +=1

    
# Time to drink some coffee!

  0%|          | 0/1000 [00:00<?, ?it/s]

>>>> policy_seq;  __str__ = [[[ 0.14181072  0.15308765  0.18365371  0.26436192  0.04178216  0.21530384]
  [ 0.14181072  0.15308765  0.18365371  0.26436192  0.04178216  0.21530384]
  [ 0.14181072  0.15308765  0.18365371  0.26436192  0.04178216  0.21530384]
  ..., 
  [ 0.14181072  0.15308765  0.18365371  0.26436192  0.04178216  0.21530384]
  [ 0.14181072  0.15308765  0.18365371  0.26436192  0.04178216  0.21530384]
  [ 0.14181072  0.15308765  0.18365371  0.26436192  0.04178216  0.21530384]]

 [[ 0.13323706  0.09744211  0.28547591  0.18515025  0.03576923  0.26292545]
  [ 0.13365018  0.09827401  0.28435853  0.18608937  0.03607522  0.26155266]
  [ 0.13347824  0.097927    0.28482494  0.18569791  0.03594777  0.26212415]
  ..., 
  [ 0.13385974  0.09869754  0.28378448  0.18656889  0.03622926  0.26086009]
  [ 0.13417967  0.09935572  0.28289679  0.18730903  0.03646975  0.25978902]
  [ 0.13428341  0.09957019  0.28260872  0.1875491   0.03654831  0.25944027]]

 [[ 0.14181072  0.15308765  0.18365371  

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (100,1) (1,25) (100,25,2) 
Apply node that caused the error: AdvancedSubtensor(Elemwise{log,no_inplace}.0, InplaceDimShuffle{0,x}.0, InplaceDimShuffle{x,0}.0, AdvancedSubtensor1.0)
Toposort index: 518
Inputs types: [TensorType(float32, 3D), TensorType(int64, col), TensorType(int64, row), TensorType(int32, 3D)]
Inputs shapes: [(100, 25, 6), (100, 1), (1, 25), (100, 25, 2)]
Inputs strides: [(600, 24, 4), (8, 8), (200, 8), (200, 8, 4)]
Inputs values: ['not shown', 'not shown', 'not shown', 'not shown']
Inputs type_num: [11, 7, 7, 5]
Outputs clients: [[Elemwise{neg,no_inplace}(AdvancedSubtensor.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/home/manatee/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py", line 501, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/manatee/anaconda3/lib/python3

In [ ]:
iters, session_rewards=zip(*sorted(rewards.items(),key=lambda pr:pr[0]))

In [ ]:
plt.plot(iters,list(map(np.mean, session_rewards)))
plt.title("Training progress")
plt.xlabel("Epoch counter")
plt.ylabel("Mean Income")
plt.show()

In [ ]:
_,_,_,_,(pool_policy,pool_V) = agent.get_sessions(
    pool.experience_replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,)

states = pool.experience_replay.observations[0].get_value().reshape((-1, 5)).T[-2:]
values = pool_V.ravel().eval()
optimal_actid = pool_policy.argmax(-1).ravel().eval()

In [ ]:
states.shape

In [ ]:
values.shape

In [ ]:
plt.scatter(
    *states,
#     c=values,
    alpha = 0.1)
plt.title("predicted state values")
plt.xlabel("previous")
plt.ylabel("current")
plt.show()

In [ ]:
obs_x, obs_y = states

for i in range(3):
    sel = (optimal_actid==i)
    plt.scatter(obs_x[sel],
                obs_y[sel],
                c=['red','blue','green'][i],
                alpha = 0.1,
                label=action_names[i])
    
plt.title("most likely action id")
plt.xlabel("previous")
plt.ylabel("current")
plt.legend(loc='best')
plt.show()

In [ ]:
#for MountainCar-v0 evaluation session is cropped to 200 ticks
untrained_reward = pool.evaluate(save_path="./records",record_video=True)

### 